In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Define paths
data_dir = 'D:/Medicinal plant dataset'

In [3]:
# Get class names from subdirectory names
class_names = sorted(os.listdir(data_dir))

In [4]:
# Print class names to check
print("Class Names:")
for class_name in class_names:
    print(class_name)

Class Names:
Aloevera
Amla
Amruthaballi
Arali
Astma_weed
Badipala
Balloon_Vine
Bamboo
Beans
Betel
Bhrami
Bringaraja
Caricature
Castor
Catharanthus
Chakte
Chilly
Citron lime (herelikai)
Coffee
Common rue(naagdalli)
Coriender
Curry
Doddpathre
Drumstick
Ekka
Eucalyptus
Ganigale
Ganike
Gasagase
Ginger
Globe Amarnath
Guava
Henna
Hibiscus
Honge
Insulin
Jackfruit
Jasmine
Kambajala
Kasambruga
Kohlrabi
Lantana
Lemon
Lemongrass
Malabar_Nut
Malabar_Spinach
Mango
Marigold
Mint
Neem
Nelavembu
Nerale
Nooni
Onion
Padri
Palak(Spinach)
Papaya
Parijatha
Pea
Pepper
Pomoegranate
Pumpkin
Raddish
Rose
Sampige
Sapota
Seethaashoka
Seethapala
Spinach1
Tamarind
Taro
Tecoma
Thumbe
Tomato
Tulsi
Turmeric
ashoka
camphor
kamakasturi
kepala


In [6]:
# Split data into training and testing sets
train_data_dir = os.path.join(data_dir, 'train')
test_data_dir = os.path.join(data_dir, 'test')

In [7]:
# Create train and test directories
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)
for class_name in class_names:
    os.makedirs(os.path.join(train_data_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_data_dir, class_name), exist_ok=True)

    class_images = os.listdir(os.path.join(data_dir, class_name))
    train_images, test_images = train_test_split(class_images, test_size=0.2, random_state=42)

    for image in train_images:
        src_path = os.path.join(data_dir, class_name, image)
        dest_path = os.path.join(train_data_dir, class_name, image)
        shutil.copy(src_path, dest_path)
    for image in test_images:
        src_path = os.path.join(data_dir, class_name, image)
        dest_path = os.path.join(test_data_dir, class_name, image)
        shutil.copy(src_path, dest_path)

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [9]:
# Data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 5487 images belonging to 80 classes.
Found 1413 images belonging to 80 classes.


In [11]:
# Build the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(len(class_names), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [13]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

Epoch 1/25
172/172 [==============================] - 740s 4s/step - loss: 2.5133 - accuracy: 0.3842 - val_loss: 10.9580 - val_accuracy: 0.0403
Epoch 2/25
172/172 [==============================] - 668s 4s/step - loss: 0.9475 - accuracy: 0.7292 - val_loss: 3.8046 - val_accuracy: 0.2958
Epoch 3/25
172/172 [==============================] - 1887s 11s/step - loss: 0.5101 - accuracy: 0.8495 - val_loss: 1.2532 - val_accuracy: 0.6348
Epoch 4/25
172/172 [==============================] - 666s 4s/step - loss: 0.3002 - accuracy: 0.9105 - val_loss: 0.9064 - val_accuracy: 0.7509
Epoch 5/25
172/172 [==============================] - 667s 4s/step - loss: 0.2481 - accuracy: 0.9211 - val_loss: 1.1917 - val_accuracy: 0.6886
Epoch 6/25
172/172 [==============================] - 701s 4s/step - loss: 0.1484 - accuracy: 0.9559 - val_loss: 1.0547 - val_accuracy: 0.7254
Epoch 7/25
172/172 [==============================] - 683s 4s/step - loss: 0.1566 - accuracy: 0.9532 - val_loss: 0.8769 - val_accuracy: 0.7

In [14]:
# Save the model
model.save('D:/model/resnet50_model.h5')

In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf


In [16]:

# Function to predict plant name from image
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array
def predict_plant_name(model, img_path, class_indices):
    img_array = load_and_preprocess_image(img_path)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]

    class_labels = {v: k for k, v in class_indices.items()}
    predicted_class_name = class_labels[predicted_class_index]
    return predicted_class_name, predicted_class_index

In [17]:
# Load the saved model
model = load_model('D:/model/resnet50_model.h5')


In [22]:
img_path = 'D:/TEST_PICS/Betel/1938.jpg'
predicted_class_name, predicted_class_index = predict_plant_name(model, img_path, train_generator.class_indices)
print(f'The predicted plant name is: {predicted_class_name}')

1/1 [==============================] - 0s 113ms/step
The predicted plant name is: Betel
